In [1]:
# Basic libraries / dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# For Modelling
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna

# For warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

In [3]:
datasets = [train, test]

In [4]:
datasets[0].head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [5]:
def drop_cols(dataset, col):
    for data in dataset:
        data.drop(col, axis = 1, inplace = True)
    return data
col = ['dw_12', 'dw_13', 'lan_13', 'pw_07', 'pw_08', 'ward', 'ADM4_PCODE']
drop_cols(datasets, col)

,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,...,pw_00,pw_01,pw_02,pw_03,pw_04,pw_05,pw_06,lat,lon,NL
0,2504.95194,8745.15151,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,0.013018,...,0.551691,0.427445,0.012457,0.001757,0.000000,0.000000,0.006649,-32.637758,23.848688,0.000000
1,2080.27718,7258.11764,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,0.022412,...,0.759594,0.227192,0.009606,0.000691,0.000000,0.000000,0.002916,-31.990536,24.555818,0.000000
2,1106.62639,5919.13170,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,0.044153,...,0.704033,0.291719,0.001996,0.002253,0.000000,0.000000,0.000000,-32.283595,24.563940,8.269556
3,2175.56096,10280.57452,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,0.101963,...,0.556172,0.439729,0.002060,0.000000,0.000661,0.000000,0.001379,-32.261612,24.542202,8.626625
4,1270.83883,6018.34202,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,0.027930,...,0.444380,0.553173,0.000787,0.000000,0.000000,0.000000,0.001660,-32.251571,24.558537,8.601754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,16076.89903,43296.51372,0.668295,0.002827,0.207749,0.028813,0.069741,0.010701,0.003941,0.000804,...,0.992238,0.004045,0.000609,0.000392,0.000291,0.000148,0.002277,-33.806524,18.496094,33.913055
1009,17470.24004,53538.12816,0.483111,0.000824,0.001189,0.000870,0.000554,0.000370,0.001954,0.088044,...,0.433723,0.346398,0.158056,0.030744,0.008313,0.000348,0.022417,-33.982120,18.673308,60.009486
1010,9467.25336,40332.11708,0.579036,0.007184,0.066458,0.001003,0.000371,0.150977,0.007699,0.099672,...,0.733120,0.215139,0.035879,0.003850,0.000990,0.000352,0.010670,-34.062704,18.767457,35.834860
1011,6111.31195,27793.92916,0.623537,0.002216,0.101188,0.001517,0.000939,0.158979,0.013516,0.076050,...,0.934310,0.022448,0.015737,0.000559,0.000000,0.000204,0.026741,-34.057772,18.487893,50.153301


In [6]:
def features(dataset):
    for data in dataset:
        data['f1'] = data.psa_00 + data.car_01 + data.stv_01 + data.lln_01
        data['f2'] = data.lgt_00 + data.pw_00 + data.pg_03 + data.dw_01
        data['ind_per_house'] = data.total_individuals / data.total_households
        data['dw'] = data.dw_00 + data.dw_01 + data.dw_02 + data.dw_03 + data.dw_04 + data.dw_05
        data['lan'] = data.lan_00 + data.lan_01 + data.lan_02 + data.lan_03 + data.lan_04 + data.lan_05
        data['pw'] = data.pw_00 + data.pw_01 + data.pw_02 + data.pw_03 + data.pw_04
        data.drop(['total_individuals', 'total_households'], axis = 1, inplace = True)
    return data
features(datasets)

,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,dw_08,dw_09,...,pw_06,lat,lon,NL,f1,f2,ind_per_house,dw,lan,pw
0,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,0.013018,0.003516,0.000000,...,0.006649,-32.637758,23.848688,0.000000,2.701130,1.557033,3.491145,0.980268,0.967260,0.993351
1,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,0.022412,0.016969,0.006702,...,0.002916,-31.990536,24.555818,0.000000,1.922068,2.040153,3.489015,0.918440,0.833398,0.997084
2,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,0.044153,0.013530,0.000000,...,0.000000,-32.283595,24.563940,8.269556,2.778162,1.688474,5.348808,0.928118,0.987500,1.000000
3,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,0.101963,0.001516,0.009203,...,0.001379,-32.261612,24.542202,8.626625,2.325306,1.579226,4.725482,0.874905,0.892816,0.998621
4,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,0.027930,0.001617,0.017343,...,0.001660,-32.251571,24.558537,8.601754,2.709457,1.398725,4.735724,0.947989,0.989536,0.998340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,0.668295,0.002827,0.207749,0.028813,0.069741,0.010701,0.003941,0.000804,0.002530,0.003673,...,0.002277,-33.806524,18.496094,33.913055,1.133560,2.610884,2.693089,0.988126,0.874785,0.997575
1009,0.483111,0.000824,0.001189,0.000870,0.000554,0.000370,0.001954,0.088044,0.416686,0.000066,...,0.022417,-33.982120,18.673308,60.009486,3.020161,1.281801,3.064533,0.486918,0.937696,0.977234
1010,0.579036,0.007184,0.066458,0.001003,0.000371,0.150977,0.007699,0.099672,0.070348,0.006363,...,0.010670,-34.062704,18.767457,35.834860,2.576032,1.708089,4.260171,0.805030,0.972096,0.988978
1011,0.623537,0.002216,0.101188,0.001517,0.000939,0.158979,0.013516,0.076050,0.001687,0.014067,...,0.026741,-34.057772,18.487893,50.153301,1.861088,1.929308,4.547948,0.888378,0.975218,0.973054


In [10]:
keep = {}
cols = train.columns.difference(['target'])
for col in cols:
    keep[col] = train.target.corr(train[col])
pd.Series(keep).sort_values(ascending = False).keys()

Index(['psa_00', 'f1', 'car_01', 'stv_01', 'lln_01', 'pg_00', 'pw_06', 'dw_01',
       'pw_02', 'pw_03', 'pw_04', 'pw_05', 'lon', 'dw', 'lan_05', 'lan_04',
       'lan_11', 'psa_04', 'lat', 'pw_01', 'lan_10', 'lan_09', 'dw_00',
       'lan_08', 'lan_02', 'lan_06', 'ind_per_house', 'lan', 'lan_07',
       'psa_02', 'dw_11', 'dw_10', 'lan_03', 'dw_05', 'dw_07', 'dw_08',
       'pg_04', 'pg_02', 'dw_02', 'pg_01', 'dw_09', 'dw_06', 'lgt_00',
       'lan_14', 'psa_03', 'lan_12', 'dw_03', 'dw_04', 'lan_01', 'pw',
       'lan_00', 'NL', 'pg_03', 'lln_00', 'f2', 'stv_00', 'car_00', 'psa_01',
       'pw_00'],
      dtype='object')

In [11]:
main = ['psa_00', 'f1', 'car_01', 'stv_01', 'lln_01', 'pg_00', 'pw_06', 'dw_01',
       'pw_02', 'pw_03', 'pw_04', 'pw_05', 'lon', 'dw', 'lan_05', 'lan_04',
       'lan_11', 'psa_04', 'lat', 'pw_01', 'lan_10', 'lan_09', 'dw_00',
       'lan_08', 'lan_02', 'lan_06', 'ind_per_house', 'lan', 'lan_07',
       'psa_02', 'dw_11', 'dw_10', 'lan_03', 'dw_05', 'dw_07', 'dw_08',
       'pg_04', 'pg_02', 'dw_02', 'pg_01', 'dw_09', 'dw_06', 'lgt_00',
       'lan_14', 'psa_03', 'lan_12', 'dw_03', 'dw_04', 'lan_01', 'pw',
       'lan_00', 'NL', 'pg_03', 'lln_00', 'f2', 'stv_00', 'car_00', 'psa_01',
       'pw_00']

In [12]:
y = train['target']
X = train[main]

In [13]:
X_ = train.drop('target', axis = 1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y, test_size = 0.2, random_state = 42)

### CatBoostRegressor

In [16]:
model_cat_ = CatBoostRegressor(verbose = 0, n_estimators = 2000) # Create the model
model_cat_.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_test_cat = model_cat_.predict(X_test)
error_test_cat = mean_squared_error(y_test, y_test_cat, squared = False)
error_test_cat

2.887011450204307

In [17]:
cat_ = CatBoostRegressor(verbose = 0, n_estimators = 2000) # Create the model
cat_.fit(X_train_, y_train_) # Train it (this syntax looks the same for all sklearn models)
y_test_cat = cat_.predict(X_test_)
error_test_cat = mean_squared_error(y_test_, y_test_cat, squared = False)
error_test_cat

2.9061375764198

In [ ]:
fold = KFold(n_splits = 5, shuffle = True, random_state = 42)
score = []
for train_index, test_index in fold.split(X):
    x_train_, x_test_ = X.iloc[train_index], X.iloc[test_index]
    y_train_, y_test_ = y.iloc[train_index], y.iloc[test_index]
    cat = CatBoostRegressor(n_estimators = 2000, verbose = 0, iterations = 955, max_depth = 6, subsample = 0.6382849390832925, 
                               learning_rate = 0.058582676139219296, min_child_samples = 38, l2_leaf_reg = 2)
    cat.fit(x_train_, y_train_) # Train it (this syntax looks the same for all sklearn models)
    y_hat = cat.predict(x_test_)
    error = mean_squared_error(y_test_, y_hat, squared = False)
    print(f'RMSE: {error}')
    score.append(error)
np.mean(score)

RMSE: 2.8840898961840318
RMSE: 3.1886095385404767
RMSE: 3.242673833877179
RMSE: 3.0998352957052644
RMSE: 3.085620200872677


3.1001657530359257

In [51]:
def objective(trial):
    model = CatBoostRegressor(
        iterations = trial.suggest_int('iterations', 400, 1000),
        max_depth = trial.suggest_int('max_depth', 3, 10),
        subsample = trial.suggest_float('subsample', 0.6, 1.0),
        learning_rate = trial.suggest_float('learning_rate', 0.03, 0.1),
        min_child_samples = trial.suggest_int('min_child_samples', 30, 50),
        l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 0.2, 5),
        logging_level = 'Silent',
        od_type = 'iter',
    )
    
    model.fit(X_train, y_train, verbose = 0, early_stopping_rounds = 200)
    cat = CatBoostRegressor(n_estimators = 2000, verbose = 0)
    y_pred = model.predict(X_test)
    error = mean_squared_error(y_test, y_pred, squared = False)
    return error

study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)

[I 2022-04-19 18:10:01,295] A new study created in memory with name: no-name-22b0fbc2-47e1-4707-8050-5b604f5275ec
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2022-04-19 18:10:09,425] Trial 0 finished with value: 3.007776688941753 and parameters: {'model__iterations': 586, 'model__max_depth': 5, 'model__subsample': 0.7730848936264454, 'model__learning_rate': 0.08486564055473661, 'model__min_child_samples': 48, 'model__l2_leaf_reg': 0}. Best is trial 0 with value: 3.007776688941753.
[I 2022-04-19 18:10:50,673] Trial 1 finished with value: 2.9759998304228645 and parameters: {'model__iterations': 710, 'model__max_depth': 8, 'model__subsample': 0.6181337169178127, 'model__learning_rate': 0.05580308482380891, 'model__min_child_samples': 48, 'model__l2_leaf_reg': 4}. Best is trial 1 with value: 2.9759998304228645.
[I 2022-04-19 18:11:11,080] Trial 2 finished with value: 2.9613368201069177 and parameters: {'model__iterations': 496, 'model

In [52]:
trial = study.best_trial
print('RMSE: {}'.format(trial.value))
print('Best Parameters: {}'.format(trial.params))
# print(trial)

RMSE: 2.8874193509552564
Best Parameters: {'model__iterations': 955, 'model__max_depth': 6, 'model__subsample': 0.6382849390832925, 'model__learning_rate': 0.058582676139219296, 'model__min_child_samples': 38, 'model__l2_leaf_reg': 2}


In [12]:
model_cat = CatBoostRegressor(verbose = 0, iterations = 955, max_depth = 6, subsample = 0.6382849390832925, 
                               learning_rate = 0.058582676139219296, min_child_samples = 38, l2_leaf_reg = 2) # Create the model
model_cat.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_test_cat = model_cat.predict(X_test)
error_test_cat = mean_squared_error(y_test, y_test_cat, squared = False)
error_test_cat

2.8874193509552564

### LGBMRegressor

In [13]:
model_lgb_ = LGBMRegressor(verbose = 0) # Create the model
model_lgb_.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_test_lgb = model_lgb_.predict(X_test)
error_test_lgb = mean_squared_error(y_test, y_test_lgb, squared = False)
error_test_lgb

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003194 seconds.
You can set `force_col_wise=true` to remove the overhead.


3.160589080311209

In [14]:
errlgb = []
fold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    param = {'objective': 'regression', 
         'metric': 'l2_root',
         'learning_rate': 0.05, 
         'num_iterations': 2000,
         'num_leaves': 35,
         'max_depth': -1,
         'min_data_in_leaf': 4,
         'bagging_fraction': 0.78,
         'bagging_freq': 1,
         'feature_fraction': 0.65}
    lgbm = LGBMRegressor(**param)
    lgbm.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose = 0, early_stopping_rounds=50)
    y_pred_lgbm = lgbm.predict(X_test)
    error = mean_squared_error(y_test, y_pred_lgbm, squared = False)
    print(f"RMSE LGBM: {error}")
errlgb.append(error)
error.mean()

[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_fraction is set=0.78, subsample=1.0 will be ignored. Current value: bagging_fraction=0.78
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
RMSE LGBM: 3.0343555726957243
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_fraction is set=0.78, subsample=1.0 will be ignored. Current value: bagging_fraction=0.78
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
RMSE LGBM: 3.10000939381

3.075678728688369

In [42]:
def objective(trial):
    model = LGBMRegressor(
        learning_rate = trial.suggest_float('learning_rate', 0.03, 0.5),
        subsample = trial.suggest_float('subsample', 0.6, 1.0),
        n_estimators = trial.suggest_int('n_estimators', 100, 500),
        num_leaves = trial.suggest_int('num_leaves', 30, 40),
        objective = 'regression', 
        metric = 'l2_root',
        num_iterations = 2000,
        max_depth = -1,
        feature_fraction = 0.65,
        verbose = 0)
    
    model.fit(X_train, y_train, verbose = 0)
    y_pred = model.predict(X_test)
    error = mean_squared_error(y_test, y_pred, squared = False)
    return error

study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 50)

[I 2022-04-19 21:38:48,769] A new study created in memory with name: no-name-3087704c-60d6-4a4e-b9fe-83efaa7e2715


[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004792 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2022-04-19 21:38:57,467] Trial 0 finished with value: 3.1805386901437864 and parameters: {'learning_rate': 0.16358122243578527, 'subsample': 0.7306755229400822, 'n_estimators': 234, 'num_leaves': 31}. Best is trial 0 with value: 3.1805386901437864.


[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002127 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2022-04-19 21:39:16,064] Trial 1 finished with value: 3.487398009528127 and parameters: {'learning_rate': 0.3533731855152852, 'subsample': 0.9812062431930717, 'n_estimators': 224, 'num_leaves': 39}. Best is trial 0 with value: 3.1805386901437864.


...

[I 2022-04-19 21:48:40,652] Trial 47 finished with value: 3.351390578065139 and parameters: {'learning_rate': 0.31724392502949084, 'subsample': 0.8582299262788565, 'n_estimators': 376, 'num_leaves': 30}. Best is trial 11 with value: 3.0020065304389596.


[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002120 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2022-04-19 21:49:04,438] Trial 48 finished with value: 3.026125562101571 and parameters: {'learning_rate': 0.030317840354062136, 'subsample': 0.7471385611779329, 'n_estimators': 418, 'num_leaves': 39}. Best is trial 11 with value: 3.0020065304389596.


[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004995 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2022-04-19 21:49:19,567] Trial 49 finished with value: 3.1019508528300883 and parameters: {'learning_rate': 0.08392888950960342, 'subsample': 0.7366263057709261, 'n_estimators': 454, 'num_leaves': 39}. Best is trial 11 with value: 3.0020065304389596.


In [43]:
trial = study.best_trial
print('RMSE: {}'.format(trial.value))
print('Best Parameters: {}'.format(trial.params))
# print(trial)

RMSE: 3.0020065304389596
Best Parameters: {'learning_rate': 0.0335840589577861, 'subsample': 0.8692402551456782, 'n_estimators': 312, 'num_leaves': 30}


In [17]:
param = {'objective': 'regression', 
         'metric': 'l2_root',
         'learning_rate': 0.032314807938206576,
         'subsample' : 0.9092367008636046,
         'num_iterations': 2000,
         'num_leaves': 36,
         'n_estimators' : 492,
         'max_depth': -1,
         'min_data_in_leaf': 4,
         'bagging_fraction': 0.78,
         'bagging_freq': 1,
         'feature_fraction': 0.65,
         'verbose' : 0}
model_lgb = LGBMRegressor(**param) # Create the model
model_lgb.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_test_lgb = model_lgb.predict(X_test)
error_test_lgb = mean_squared_error(y_test, y_test_lgb, squared = False)
error_test_lgb

[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_fraction is set=0.78, subsample=0.9092367008636046 will be ignored. Current value: bagging_fraction=0.78
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.65, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] bagging_fraction is set=0.78, subsample=0.9092367008636046 will be ignored. Current value: bagging_fraction=0.78
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Aut

2.982444358919984

### XGBRegressor

In [20]:
model_xgb = XGBRegressor(verbose = 0) # Create the model
model_xgb.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_test_xgb = model_xgb.predict(X_test)
error_test_xgb = mean_squared_error(y_test, y_test_xgb, squared = False)
error_test_xgb

[21:06:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




3.3804380442948894

In [29]:
errxgb = []
fold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model_xgb = XGBRegressor() # Create the model
    model_xgb.fit(X_train, y_train, eval_set = [(X_test, y_test)], verbose = 0, early_stopping_rounds=50) # Train it (this syntax looks the same for all sklearn models)
    y_hat = model_xgb.predict(X_test)
    error_test = mean_squared_error(y_test, y_hat, squared = False)
    errxgb.append(error_test)
    print(f"RMSE LGBM: {error_test}")
errxgb.append(error)
print(f'CV_RMSE: {np.mean(errxgb)}')

RMSE LGBM: 3.374716286406206
RMSE LGBM: 3.6055284090777873
RMSE LGBM: 3.451197287238
RMSE LGBM: 3.490860925072187
RMSE LGBM: 3.4824971141632948
CV_RMSE: 3.4134131251076405


In [37]:
def run(trial):
    model = XGBRegressor(
        learning_rate = trial.suggest_float('learning_rate', 1e-2, 0.25, log = True),
        reg_lambda = trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
        reg_alpha = trial.suggest_loguniform('reg_alpha', 1e-8, 100.0),
        subsample = trial.suggest_float('subsample', 0.1, 1.0),
        colsample_bylevel = trial.suggest_float('colsample_bylevel', 0.1, 1.0),
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0),
        max_depth = trial.suggest_int('max_depth', 3, 10),
        gpu_id = 1,
        n_estimators = 2000)
    
    model.fit(X_train, y_train, verbose = 1000, early_stopping_rounds = 300, eval_set = [(X_test, y_test)])
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared = False)
    return rmse

study = optuna.create_study(direction = 'minimize')
study.optimize(run, n_trials = 50)

[I 2022-04-19 21:16:42,628] A new study created in memory with name: no-name-bfdf503a-615b-45ca-8f12-767106c63a3c


[21:16:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[0]	validation_0-rmse:23.54061
[387]	validation_0-rmse:3.29474


[I 2022-04-19 21:16:46,846] Trial 0 finished with value: 3.27225797683086 and parameters: {'learning_rate': 0.09528814491404994, 'reg_lambda': 3.6031892433748592e-06, 'reg_alpha': 0.35397151898627405, 'subsample': 0.31022823412861833, 'colsample_bylevel': 0.9148542977342179, 'colsample_bytree': 0.5143269313437568, 'max_depth': 9}. Best is trial 0 with value: 3.27225797683086.


[21:16:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[0]	validation_0-rmse:25.42235
[1000]	validation_0-rmse:3.17885
[1999]	validation_0-rmse:3.17785


[I 2022-04-19 21:17:05,056] Trial 1 finished with value: 3.177852016270951 and parameters: {'learning_rate': 0.021411920246858885, 'reg_lambda': 0.044939223887503536, 'reg_alpha': 2.6592228830276747e-08, 'subsample': 0.935392688595594, 'colsample_bylevel': 0.3374677348384588, 'colsample_bytree': 0.32475369175023705, 'max_depth': 8}. Best is trial 1 with value: 3.177852016270951.


...

[I 2022-04-19 21:33:03,655] Trial 47 finished with value: 2.9757104828479686 and parameters: {'learning_rate': 0.01034881315666673, 'reg_lambda': 1.7539863778746273e-05, 'reg_alpha': 7.875565079894343e-08, 'subsample': 0.4953835201159814, 'colsample_bylevel': 0.453666791043669, 'colsample_bytree': 0.8326064914150392, 'max_depth': 5}. Best is trial 43 with value: 2.947958452285846.


[21:33:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[0]	validation_0-rmse:25.65905
[1000]	validation_0-rmse:2.98365
[1999]	validation_0-rmse:2.97047


[I 2022-04-19 21:33:32,292] Trial 48 finished with value: 2.9704371909906895 and parameters: {'learning_rate': 0.012137853742959247, 'reg_lambda': 4.846703733926485e-06, 'reg_alpha': 3.669245792042979e-07, 'subsample': 0.8401046927142767, 'colsample_bylevel': 0.2869110507676956, 'colsample_bytree': 0.868570643752211, 'max_depth': 7}. Best is trial 43 with value: 2.947958452285846.


[21:33:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:223: No visible GPU is found, setting `gpu_id` to -1
[0]	validation_0-rmse:25.58576
[1000]	validation_0-rmse:3.13122
[1999]	validation_0-rmse:3.03540


[I 2022-04-19 21:33:51,264] Trial 49 finished with value: 3.0341416645312314 and parameters: {'learning_rate': 0.015186407636673272, 'reg_lambda': 7.543018473740945e-07, 'reg_alpha': 5.533422807850566e-08, 'subsample': 0.7605508883137756, 'colsample_bylevel': 0.3653376325526461, 'colsample_bytree': 0.9014287937655918, 'max_depth': 3}. Best is trial 43 with value: 2.947958452285846.


In [38]:
trial = study.best_trial
print('RMSE: {}'.format(trial.value))
print('Best Parameters: {}'.format(trial.params))
# print(trial)

RMSE: 2.947958452285846
Best Parameters: {'learning_rate': 0.013405751761321347, 'reg_lambda': 1.912400936649247e-05, 'reg_alpha': 3.007881279310061e-07, 'subsample': 0.8418942683145513, 'colsample_bylevel': 0.3756176572022918, 'colsample_bytree': 0.8408725111250259, 'max_depth': 5}


In [41]:
params: {'learning_rate': 0.013405751761321347,
         'reg_lambda': 1.912400936649247e-05,
         'reg_alpha': 3.007881279310061e-07, 
         'subsample': 0.8418942683145513, 
         'colsample_bylevel': 0.3756176572022918, 
         'colsample_bytree': 8408725111250259, 
         'max_depth': 5,
         'gpu_id' : 1,
         'n_estimators' : 2000}
model_xgb = XGBRegressor(**params) # Create the model
model_xgb.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)
y_hat_xgb = model_xgb.predict(X_test)
error_xgb = mean_squared_error(y_test, y_hat_xgb, squared = False)
error_xgb

[21:38:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




6.846033245739254

In [45]:
X_test__ = test[main]

In [46]:
a = cat.predict(X_test__).round(6)
b = model_cat_.predict(X_test__).round(6)
c = model_lgb_.predict(X_test__).round(6)
d = lgbm.predict(X_test__).round(6)

In [47]:
test_pred_1 = (a + b + c + d) / 4

In [48]:
test_pred_2 = a*0.15 + b*0.4 + c*0.3 + d*0.15

In [49]:
ss.target = test_pred_1
ss.to_csv('model_1.csv', index = False)

In [50]:
ss.target = test_pred_2
ss.to_csv('model_11.csv', index = False)